In [1]:
import pandas as pd
import numpy as np
import feather
#import statsmodels.formula.api as smf
#from statsmodels.iolib.summary2 import summary_col
#import seaborn as sns
#import matplotlib.pyplot as plt
import os
#import altair as alt
from pathlib import Path

## Import data 

In [11]:
datadir = "../input"
Path(datadir).mkdir(parents=True, exist_ok=True) # Create if doesn't exist

In [12]:
os.chdir(datadir)
patents_df = pd.read_feather("solar_dataset_all_renewables") 
patents_df.head()

,appln_id,appln_auth,appln_kind,appln_filing_date,appln_filing_year,receiving_office,earliest_filing_year,earliest_filing_id,earliest_publn_year,granted,...,nb_citing_docdb_fam,ipc_class_symbol,ipc_class_level,ipc_value,fossil_fuel,person_ctry_code,cpc_class_symbol,cpc_gener_auth,cpc_position,cpc_value
0,66,EP,A,2004-03-02,2004,,2003,905739597,2005,Y,...,14,F27B5/04,A,I,1.0,FR,None,None,None,None
1,103,EP,A,2006-04-13,2006,,2005,6502820,2007,N,...,4,F01K21/04,A,I,1.0,CH,None,None,None,None
2,127,EP,A,2006-10-02,2006,,2005,1109097,2008,Y,...,10,F27D3/02,A,I,1.0,AT,None,None,None,None
3,148,EP,A,2007-09-27,2007,,2006,15054845,2008,Y,...,11,F27B17/02,A,I,1.0,AT,None,None,None,None
4,256,EP,A,2008-06-06,2008,,2007,55458188,2009,Y,...,5,F23N1/02,A,I,1.0,DE,None,None,None,None


In [13]:
patents_df["country_code"] = patents_df['person_ctry_code']
# patents_df["country_code"] = patents_df["appln_auth"] # cover all observations but assign the authority country not the innovator country. This mostly concerns Chinese authority applications. 

In [14]:
# Classify patents as fossil fuel or renewables

patents_df.loc[patents_df["cpc_class_symbol"].str.startswith("Y02E", na=False), 'cpc_renewables'] = 1
patents_df.loc[~patents_df["cpc_class_symbol"].str.startswith("Y02E", na=False), 'cpc_renewables'] = 0

# Solar only:
patents_df.loc[patents_df["cpc_class_symbol"].str.startswith("Y02E10/5", na=False), 'cpc_solar'] = 1
patents_df.loc[~patents_df["cpc_class_symbol"].str.startswith("Y02E10/5", na=False), 'cpc_solar'] = 0

In [15]:
patents_df=patents_df.dropna(subset=["country_code"]).reset_index()

In [16]:
# List all countries patenting 
patents_df["country_code"].unique()
list_countries = patents_df["country_code"].unique().tolist()

## Count patents 

In [17]:
### Absolute count ###

def absolute_count(df, list_countries):

    ## fossil fuel ##
    patents_year = df[df.fossil_fuel == 1].groupby('earliest_filing_year')['docdb_family_id'].nunique().to_frame(name = "Nb_ff_energy_innov_per_year_all") # Transform the series that groupby creates to a dataframe

    # countries #
    dict = {} 
    for country in list_countries:
        name = "Nb_ff_energy_innov_per_year_" + country
        value = df[(df.fossil_fuel == 1) & (df.country_code == country)].groupby('earliest_filing_year')['docdb_family_id'].nunique() 
        dict["Nb_ff_energy_innov_per_year_" + country] = value
        patents_year[name] = dict[name] 


    ## Renewables panel ##
    patents_year["Nb_renewables_energy_innov_per_year_all"] = df[df["cpc_renewables"] == 1].groupby('earliest_filing_year')['docdb_family_id'].nunique()

    # countries #
    # by family
    dict = {} 
    for country in list_countries:
        name = "Nb_renewables_energy_innov_per_year_" + country
        value = df[(df["cpc_renewables"] == 1) & (df.country_code == country)].groupby('earliest_filing_year')['docdb_family_id'].nunique()
        dict["Nb_renewables_energy_innov_per_year_" + country] = value
        patents_year[name] = dict[name] 

    
    return patents_year

In [25]:
# Save to feather 
os.chdir(datadir)
absolute_count(patents_df, list_countries).reset_index().to_feather("patents_df")
df2 = absolute_count(patents_df, list_countries).reset_index()

/var/folders/c9/20n96n6900d0kkmvh520dmdsj482n4/T/ipykernel_11029/1853555206.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  patents_year[name] = dict[name]
/var/folders/c9/20n96n6900d0kkmvh520dmdsj482n4/T/ipykernel_11029/1853555206.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  patents_year[name] = dict[name]
/var/folders/c9/20n96n6900d0kkmvh520dmdsj482n4/T/ipykernel_11029/1853555206.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [36]:
# Melt DataFrame
df_melted = df2.melt(id_vars='earliest_filing_year', var_name='Country_Type', value_name=['Nb_ff_energy_innov_per_year', 'Nb_renewables_energy_innov_per_year'])

# Extract country from Country_Type column
df_melted['Country'] = df_melted['Country_Type'].str.slice(start=-2)
df_melted['Type'] = df_melted['Country_Type'].str.slice(stop=-3)  # -3 to remove _CN like part

# Remove redundant column
df_melted.drop('Country_Type', axis=1, inplace=True)

# Reorder columns
df_melted = df_melted[['earliest_filing_year', 'Country', 'Type', 'Nb_ff_energy_innov_per_year', 'Nb_renewables_energy_innov_per_year']]

TypeError: unhashable type: 'list'

In [61]:
# Melt DataFrame
df_melted = df2.melt(id_vars='earliest_filing_year', var_name='Type_Country', value_name='Number_of_innovations')

# Extract country and type of innovation from Type_Country column
df_melted['Country'] = df_melted['Type_Country'].str[-2:]
df_melted['Type'] = df_melted['Type_Country'].str.split('_').str[1]

# Pivot the DataFrame to create separate columns for each type of innovation
df_pivot = df_melted.pivot_table(index=['earliest_filing_year', 'Country'], columns='Type', values='Number_of_innovations').reset_index()

# Flatten the multi-level column index
df_pivot.columns = df_pivot.columns.get_level_values(0)

# # Optionally, rename the columns if necessary
# df_pivot.rename(columns={'ff_energy': 'Nb_ff_energy_innov_per_year', 'renewables': 'Nb_renewables_energy_innov_per_year'}, inplace=True)

In [54]:
df_pivot.tail(50)

Type,earliest_filing_year,Country,ff,renewables
1585,2018,PL,96.0,NaN
1586,2018,RO,5.0,NaN
1587,2018,SI,4.0,NaN
1588,2018,UA,8.0,NaN
1589,2018,CL,3.0,NaN
1590,2018,CR,1.0,NaN
1591,2018,GR,2.0,NaN
1592,2018,KZ,1.0,NaN
1593,2018,MD,3.0,NaN
1594,2018,PT,3.0,NaN


In [62]:
# Keep year 2015 
df_pivot_2015 = df_pivot[df_pivot['earliest_filing_year'] == 2015]

In [75]:
# Save as feather 
df_pivot_2015.reset_index(drop = True, inplace = True)
os.chdir(datadir)
df_pivot_2015.to_feather("df_2015")